In [3]:
import math
from matplotlib import pyplot as plt
import numpy as np
from cmath import sqrt
import random as rn
import numpy as np
from numpy.random import choice as np_choice

In [4]:
# Matriz de distancias entre ciudades
distanceMatrix = np.genfromtxt('gtp4datos/gr17.csv', delimiter=',')
# relleno la diagonal con infinito para que no haya división por cero
np.fill_diagonal(distanceMatrix, np.inf)

In [11]:
class AntColony(object):

    def __init__(self, distances, n_ants, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponenet on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1
        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        # self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.pheromone = np.random.rand(self.distances.shape[0],self.distances.shape[1])
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def spread_pheromone(self,all_paths):
        for path,lenght in all_paths:
            for move in path:
                self.pheromone[move] += 1.0 / lenght
    def run(self,start):
        it=0
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        same_paths=False
        while (same_paths==False or it<2):
            all_paths = self.gen_all_paths(start)
            self.pheromone = self.pheromone * self.decay
            self.spread_pheromone(all_paths)
            all_lenghts=[]

            shortest_path = min(all_paths, key=lambda x: x[1])
            print(f'shortest path of iteration {it}:  {shortest_path}')
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path  
            it=it+1
            same_paths=True
            paths_of_paths=[]
            for p,l in all_paths:
                paths_of_paths.append(p)
            # print('ahhhhhhhhhhhhhhhhhhhhhhhhhh' , sort.(all_paths, key=lambda x: x[1])
            for i in paths_of_paths:
                for j in paths_of_paths:
                    if (i!=j):
                        same_paths=False
        print(f'the all time shortest of them all is....... {all_time_shortest_path}')
        return(all_time_shortest_path)


    def gen_all_paths(self,start):
        all_paths = []
        for i in range(self.n_ants):
            # start= np.random.randint(len(self.distances))
            path = self.gen_path(start)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths 
    
    def gen_path(self, start):
        path = []
        visited = []
        visited.append(start)
        # path.append(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.append(move)
        path.append((prev,start)) # going back to where we started    
        print('visitados:  ',visited)
        return path
    
    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[visited] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move       


In [12]:
ant_colony = AntColony(distanceMatrix, 15, 0.95, alpha=1, beta=2)
len(ant_colony.distances)
all_time_shortest_path=ant_colony.run(0)


visitados:   [0, 12, 7, 6, 3, 10, 13, 14, 2, 5, 16, 15, 11, 8, 4, 9, 1]
visitados:   [0, 7, 6, 16, 5, 12, 3, 15, 11, 4, 13, 2, 1, 8, 14, 9, 10]
visitados:   [0, 12, 2, 14, 13, 5, 6, 16, 7, 3, 8, 15, 11, 4, 10, 1, 9]
visitados:   [0, 12, 7, 3, 6, 16, 5, 13, 14, 2, 10, 4, 15, 11, 8, 1, 9]
visitados:   [0, 12, 3, 15, 16, 2, 14, 13, 10, 6, 7, 5, 8, 11, 4, 9, 1]
visitados:   [0, 6, 7, 16, 5, 14, 2, 10, 13, 4, 3, 12, 11, 8, 15, 1, 9]
visitados:   [0, 12, 16, 6, 7, 13, 14, 2, 10, 4, 3, 8, 11, 1, 9, 5, 15]
visitados:   [0, 6, 7, 16, 5, 14, 13, 2, 10, 4, 1, 3, 8, 11, 12, 15, 9]
visitados:   [0, 6, 16, 12, 14, 4, 9, 8, 11, 3, 7, 5, 13, 15, 1, 10, 2]
visitados:   [0, 6, 3, 12, 16, 5, 7, 8, 15, 10, 4, 9, 13, 11, 2, 14, 1]
visitados:   [0, 8, 2, 14, 13, 12, 3, 7, 6, 16, 5, 1, 4, 9, 10, 15, 11]
visitados:   [0, 7, 6, 16, 3, 12, 13, 2, 14, 4, 10, 1, 8, 11, 15, 5, 9]
visitados:   [0, 3, 5, 2, 14, 13, 16, 6, 12, 7, 8, 11, 1, 4, 10, 15, 9]
visitados:   [0, 3, 12, 16, 7, 6, 1, 4, 13, 14, 2, 10, 11, 5, 9,